In [12]:
import GPy
import numpy as np
import sys
sys.path.append("../../")
from get_data import Data
import err

    
dim_set=[(1,2),(0,1,2)]         

for dim in dim_set:
    f=open("../kernel_add/"+str(dim)+".txt","w")
    train_num_set=[52]
    
    kernel_set=[
                GPy.kern.Bias(input_dim=len(dim)),
                GPy.kern.Exponential(input_dim=len(dim)),          
                GPy.kern.Linear(input_dim=len(dim)),
                GPy.kern.Matern32(input_dim=len(dim)),
                GPy.kern.Matern52(input_dim=len(dim)),     
                GPy.kern.MLP(input_dim=len(dim)),
                GPy.kern.Poly(input_dim=len(dim)),
                GPy.kern.RBF(input_dim=len(dim)),
                ]
    f.write("kernel\t\t\t\ttrain_rmse\ttest_rmse\tavg_err\t\tlog-likelihood\tmodel")    
    f.write("\n\n")  
    
    for i in range(len(kernel_set)):
        for j in range(i+1,len(kernel_set)):
            kernel_set[i]
            kernel_set[j]
    
            f.write("%-8s\t"%type(kernel_set[i]).__name__)
            f.write("+  ")
            f.write("%-10s\t"%type(kernel_set[j]).__name__)

            
            kernel=kernel_set[i]+kernel_set[j]

            train_num=52
            data=Data("../../data/data.txt",dim,train_num)

            data.train.mean_and_std()
            X=data.train.mean_and_std_X
            Y=data.train.mean_and_std_Y

            #构建高斯过程模型#
            
            #kernel=GPy.kern.Matern52(input_dim=len(dim))
            model=GPy.models.GPRegression(X,Y,kernel)
            #model.optimize(messages=True,max_f_eval = 1000)
            model.optimize(max_f_eval = 1000)


            # test 使用 train的数据归一化
            data.test.mean_and_std(data.train.mean_x,data.train.mean_y,
                                data.train.std_x,data.train.std_y)

            mean_train,var_train=model.predict(data.train.mean_and_std_X)
            #train 上的mean_train和var_train

            mean_test,var_test=model.predict(data.test.mean_and_std_X)
            # test 上的 mean_test 和 var_test

            #反归一化回原值
            data.train.re_mean_and_std(mean_train)
            data.test.re_mean_and_std(mean_test)

            
            
            
            #f.write("%-10s\t"%type(kernel).__name__)
            f.write("%.10f\t"%err.calc_rmse(data.train.Y,data.train.predict_Y))
            f.write("%.10f\t"%err.calc_rmse(data.test.Y,data.test.predict_Y))
            f.write("%.10f\t"%err.calc_err_rate(data.test.Y,data.test.predict_Y))
            f.write("%.10f\t"%model.log_likelihood())
            
            for a,b in zip(model.parameter_names(),model.param_array):
                f.write(str(a)+": ")
                f.write(str(b))
                f.write("\t")
            
            f.write("\n")
            
            #print(model)
            
            
    f.close()
